<a href="https://colab.research.google.com/github/u-masao/vector-search-tutorial-for-japanese/blob/main/Session04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語のためのベクトル検索ハンズオン: Session 4

## はじめに

『日本語のためのベクトル検索ハンズオン』の Session 4 では、Qdrant の実践的な使い方を確認します。

[Livedoor New コーパス](https://www.rondhuit.com/download.html) に収録された日本語のニュース記事を Qdrant でベクトル検索します。

## 環境構築


### Google Colab 向け Docker インストール、Qdrant 起動

#### udocker のセットアップ

In [1]:
# Copyright 2024 Drengskapur
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# @title {display-mode:"form"}
# @markdown <br/><br/><center><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/docker.svg" height="150"><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/colab.svg" height="150"></center><br/>
# @markdown <center><h1>Docker in Colab</h1></center><center>github.com/drengskapur/docker-in-colab<br/><br/><br/><b>udocker("run hello-world")</b></center><br/>
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

Docker-in-Colab 1.1.0

Usage:     udocker("--help")
Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples


#### Qdrant *コンテナを起動*

In [2]:
udocker("run -p 127.0.0.1:6333:6333 -p 127.0.0.1:6334:6334 qdrant/qdrant")
# 初回実行時は正常に終了しないことがあります。しばらくして応答がなければランタイムをリセットしてください。
# 「2024-05-24T13:42:54.582427Z ERROR qdrant: Error while starting REST server: Address already in use (os error 98) 」等とエラーメッセージが表示される場合は、既に Qdrant のコンテナが実行されています

user@pc$ udocker run -p 127.0.0.1:6333:6333 -p 127.0.0.1:6334:6334 qdrant/qdrant
 
 ****************************************************************************** 
 *                                                                            * 
 *               STARTING 47b04aae-a16b-3fcd-bfb6-0fcbe7b6b4a4                * 
 *                                                                            * 
 ****************************************************************************** 
 executing: entrypoint.sh
           _                 _    
  __ _  __| |_ __ __ _ _ __ | |_  
 / _` |/ _` | '__/ _` | '_ \| __| 
| (_| | (_| | | | (_| | | | | |_  
 \__, |\__,_|_|  \__,_|_| |_|\__| 
    |_|                           

Version: 1.9.2, build: 34f7f8ec
Access web UI at http://localhost:6333/dashboard

2024-05-24T14:25:05.812905Z  INFO storage::content_manager::consensus::persistent: Initializing new raft state at ./storage/raft_state.json    
2024-05-24T14:25:05.852229Z  INFO qdrant: Distrib

###  SageMaker 向け docker インストール、Qdrant 起動

SageMaker 等、直接 Docker を実行できる環境の場合は下のセルを実行して Qdrant コンテナを起動します。

In [3]:
# 実行する際はコメントを外してください
# !docker run -d -p 127.0.0.1:6333:6333 -p 127.0.0.1:6334:6334 qdrant/qdrant

## パッケージのインストールと読み込み

In [4]:
!pip install -q sentence-transformers datasets qdrant-client

In [5]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

## Qdrant を利用した日本語テキストのベクトル検索

以下を実行します。

- ベクトル作成
  - データ取得
  - テキスト作成
  - 埋め込み計算
- Qdrant Client を初期化
- Qdrant にデータを登録
  - コレクションを初期化(Cosine計算モード)
  - ポイント(アイテム)を登録
    - ベクトル、ペイロード、インデックス
- ベクトル検索
  - ベクトル検索のクエリ例
  - メタデータフィルタリングとベクトル検索を併用するクエリ例

### データの読み込みと埋め込みベクトルの計算

In [6]:
# Hugging Face からデータセットを取得
# データセットの説明 → https://huggingface.co/datasets/sbintuitions/JMTEB
dataset = load_dataset("sbintuitions/JMTEB", name="livedoor_news")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for sbintuitions/JMTEB contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sbintuitions/JMTEB
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  wa

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['url', 'timestamp', 'title', 'text', 'label'],
        num_rows: 5163
    })
    validation: Dataset({
        features: ['url', 'timestamp', 'title', 'text', 'label'],
        num_rows: 1106
    })
    test: Dataset({
        features: ['url', 'timestamp', 'title', 'text', 'label'],
        num_rows: 1107
    })
})

In [8]:
dataset['train'].features

{'url': Value(dtype='string', id=None),
 'timestamp': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['dokujo-tsushin', 'it-life-hack', 'kaden-channel', 'livedoor-homme', 'movie-enter', 'peachy', 'smax', 'sports-watch', 'topic-news'], id=None)}

In [9]:
# データセットからデータフレームを作成(100件をサンプリング)
text_df  = pd.DataFrame(dataset['train']).sample(100, random_state = 12345)  # 100件をランダムに抽出

# ラベル名を作成
label_names = dataset['train'].features['label'].names
text_df['label_name'] = text_df['label'].apply(lambda x: label_names[x])

# URL 順に表示
text_df.sort_values('url')

,url,timestamp,title,text,label,label_name
4584,http://news.livedoor.com/article/detail/4429538/,2009-11-09T13:00:00+0900,倖田來未と「ル パティシエ タカギ」、クリスマスケーキで夢のコラボ！,\n「上と下で味が違う、1つで2度美味しい、そしてアラザンやリボンでオリジナルなデコレーショ...,5,peachy
95,http://news.livedoor.com/article/detail/4499588/,2009-12-11T19:40:00+0900,聖なる夜に最高のおもてなしを…。リゾート気分でディナーを堪能,\nThe Terrace ザ・テラス\nお客様に喜んでもらうことをモットーに、常に、最高の...,5,peachy
3067,http://news.livedoor.com/article/detail/4684025/,2010-03-27T23:00:00+0900,東京モーターサイクルショー2010／コンパニオン写真集,\n会場には国内外の二輪車メーカーなどのブースが立ち並び、コンセプトバイクの発表や各社が用意...,3,livedoor-homme
4583,http://news.livedoor.com/article/detail/4745261/,2010-04-29T07:30:00+0900,【Sports Watch】鬼嫁・新山千春“ゴミが私の手にやってくるって考えられない”,\n男性視点で見れば、非常に気の毒なエピソードが次々に寄せられる同放送だったが、中には（例え...,7,sports-watch
4043,http://news.livedoor.com/article/detail/4897873/,2010-07-22T10:45:00+0900,本物にこだわる独女に “国連が認めた”基礎化粧品,\n普段より一層お肌のお手入れを丁寧に、そして、独女世代ならば本当に良い物を吟味して使用した...,0,dokujo-tsushin
...,...,...,...,...,...,...
1372,http://news.livedoor.com/article/detail/6865979/,2012-08-18T11:55:00+0900,自分が選んだニュースにコメントが！シェアで変わるITニュース活用法【ITニュースで目指せ情報通】,\n特にIT市場は、最新のガジェット、アプリ、サービス、テクノロジーがあふれています。\nそ...,1,it-life-hack
3345,http://news.livedoor.com/article/detail/6872658/,2012-08-21T09:55:00+0900,パチンコ店でも通話がクリア！電話としてのau最新スマホの音声を徹底チェック,\nカメラ、インターネット、ナビといった楽しい機能から、キャッシュレス電子決済や電話、テレビ...,1,it-life-hack
1673,http://news.livedoor.com/article/detail/6874095/,2012-08-21T15:55:00+0900,テレコムスクエア、LTE対応のカナダ専用4GWi-Fiルータのレンタルサービスを開始！料金は...,\nテレコムスクエアは21日、日本国内で貸し出しサービスを行うレンタルWi-Fi ルータ「W...,6,smax
4892,http://news.livedoor.com/article/detail/6880200/,2012-08-23T09:00:00+0900,ガガを聴きながら毎朝歯磨き！ レディー・ガガ シンギングトゥースブラシ独占先行販売,\nこうした中には好きなミュージシャンの歌を聴きながら歯が磨ければ、気分もすっきりして1日を...,1,it-life-hack


In [10]:
# 埋め込み用のテキストを作成する関数
def build_concat_text(row):

    # テンプレート
    return f'''
        # タイトル

        {row['title']}

        ## ニュースカテゴリ

        {row['label_name']}

        ## 記事

        {row['text']}
    '''

# 埋め込み対象テキストを作成
text_df['embed_text'] = text_df.apply(build_concat_text, axis=1)

# ライセンス表記の行を削除
text_df = text_df[~text_df['url'].str.startswith('このディレクトリにあるすべての記事ファイル')]

# URL からアイテムIDを計算
text_df['id'] = text_df['url'].replace('.*detail/([0-9]*)/',r'\1',regex=True).astype(int)

# タイトルの長さを計算(フィルタリング用)
text_df['title_length'] = text_df['title'].str.len()

# 記事本体の長さを計算(フィルタリング用)
text_df['text_length'] = text_df['text'].str.len()

# URL 順で表示
text_df.sort_values('id')

,url,timestamp,title,text,label,label_name,embed_text,id,title_length,text_length
4584,http://news.livedoor.com/article/detail/4429538/,2009-11-09T13:00:00+0900,倖田來未と「ル パティシエ タカギ」、クリスマスケーキで夢のコラボ！,\n「上と下で味が違う、1つで2度美味しい、そしてアラザンやリボンでオリジナルなデコレーショ...,5,peachy,\n # タイトル\n\n 倖田來未と「ル パティシエ タカギ」...,4429538,34,524
95,http://news.livedoor.com/article/detail/4499588/,2009-12-11T19:40:00+0900,聖なる夜に最高のおもてなしを…。リゾート気分でディナーを堪能,\nThe Terrace ザ・テラス\nお客様に喜んでもらうことをモットーに、常に、最高の...,5,peachy,\n # タイトル\n\n 聖なる夜に最高のおもてなしを…。リゾ...,4499588,30,679
3067,http://news.livedoor.com/article/detail/4684025/,2010-03-27T23:00:00+0900,東京モーターサイクルショー2010／コンパニオン写真集,\n会場には国内外の二輪車メーカーなどのブースが立ち並び、コンセプトバイクの発表や各社が用意...,3,livedoor-homme,\n # タイトル\n\n 東京モーターサイクルショー2010／...,4684025,27,752
4583,http://news.livedoor.com/article/detail/4745261/,2010-04-29T07:30:00+0900,【Sports Watch】鬼嫁・新山千春“ゴミが私の手にやってくるって考えられない”,\n男性視点で見れば、非常に気の毒なエピソードが次々に寄せられる同放送だったが、中には（例え...,7,sports-watch,\n # タイトル\n\n 【Sports Watch】鬼嫁・新...,4745261,43,527
4043,http://news.livedoor.com/article/detail/4897873/,2010-07-22T10:45:00+0900,本物にこだわる独女に “国連が認めた”基礎化粧品,\n普段より一層お肌のお手入れを丁寧に、そして、独女世代ならば本当に良い物を吟味して使用した...,0,dokujo-tsushin,\n # タイトル\n\n 本物にこだわる独女に “国連が認めた...,4897873,24,977
...,...,...,...,...,...,...,...,...,...,...
1372,http://news.livedoor.com/article/detail/6865979/,2012-08-18T11:55:00+0900,自分が選んだニュースにコメントが！シェアで変わるITニュース活用法【ITニュースで目指せ情報通】,\n特にIT市場は、最新のガジェット、アプリ、サービス、テクノロジーがあふれています。\nそ...,1,it-life-hack,\n # タイトル\n\n 自分が選んだニュースにコメントが！シ...,6865979,48,2413
3345,http://news.livedoor.com/article/detail/6872658/,2012-08-21T09:55:00+0900,パチンコ店でも通話がクリア！電話としてのau最新スマホの音声を徹底チェック,\nカメラ、インターネット、ナビといった楽しい機能から、キャッシュレス電子決済や電話、テレビ...,1,it-life-hack,\n # タイトル\n\n パチンコ店でも通話がクリア！電話とし...,6872658,37,2436
1673,http://news.livedoor.com/article/detail/6874095/,2012-08-21T15:55:00+0900,テレコムスクエア、LTE対応のカナダ専用4GWi-Fiルータのレンタルサービスを開始！料金は...,\nテレコムスクエアは21日、日本国内で貸し出しサービスを行うレンタルWi-Fi ルータ「W...,6,smax,\n # タイトル\n\n テレコムスクエア、LTE対応のカナダ...,6874095,62,1699
4892,http://news.livedoor.com/article/detail/6880200/,2012-08-23T09:00:00+0900,ガガを聴きながら毎朝歯磨き！ レディー・ガガ シンギングトゥースブラシ独占先行販売,\nこうした中には好きなミュージシャンの歌を聴きながら歯が磨ければ、気分もすっきりして1日を...,1,it-life-hack,\n # タイトル\n\n ガガを聴きながら毎朝歯磨き！ レディ...,6880200,41,1060


In [11]:
# 統計情報を表示
text_df.describe()

,label,id,title_length,text_length
count,100.00000,1.000000e+02,100.000000,100.00000
mean,3.94000,6.124321e+06,36.380000,1034.84000
std,2.80627,6.292525e+05,12.221673,612.07555
min,0.00000,4.429538e+06,12.000000,134.00000
25%,1.00000,5.809153e+06,27.000000,601.75000
50%,4.00000,6.325662e+06,37.000000,865.50000
75%,7.00000,6.655402e+06,44.250000,1417.75000
max,8.00000,6.903121e+06,73.000000,4188.00000


In [12]:
# 埋め込みモデルを初期化
#   日本語の埋め込みで評判が良いモデルを利用
#   https://huggingface.co/intfloat/multilingual-e5-small
model = SentenceTransformer("intfloat/multilingual-e5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
# 埋め込みモデルの出力次元酢を取得
embedding_dim = model.get_sentence_embedding_dimension()
print("Embedding dimension:", embedding_dim)

Embedding dimension: 384


In [14]:
# タイトルと本文を結合したテキストをベクトル化
# multilingual-e5-small では最初の 512 トークン分までを反映
vectors = model.encode(text_df['embed_text'].values, normalize_embeddings=False)

In [15]:
# 埋め込みベクトルをファイルに保存
# np.save("vectors.npy", vectors)

In [16]:
# 埋め込みベクトルをロード
# vectors = np.load("vectors.npy")

In [17]:
# 取得した埋め込みベクトルの概要
print("ndarray.shape:",vectors.shape)
print("各ベクトルの長さ(L2ノルム)の算術平均:",np.linalg.norm(vectors, ord=2, axis=1).mean())

ndarray.shape: (100, 384)
各ベクトルの長さ(L2ノルム)の算術平均: 1.0


In [18]:
# サンプル表示
vectors[0]

array([ 0.08066503, -0.00523379, -0.09744685, -0.01677839,  0.07524786,
       -0.04739355,  0.03523386,  0.01692776,  0.05449464,  0.04589533,
        0.07023565,  0.00129315,  0.04883071, -0.02052256, -0.05360262,
        0.06391852,  0.05331073, -0.04042253, -0.04317233, -0.06652067,
        0.03185656,  0.01994527, -0.02091336,  0.02510303,  0.02901173,
        0.05440372, -0.01039908,  0.0743029 ,  0.06434262, -0.05368255,
       -0.08908329, -0.02670831,  0.08869047, -0.03918426,  0.06906943,
        0.04412019, -0.08337806, -0.0278442 ,  0.03416985, -0.02788029,
       -0.06567191,  0.00523806,  0.03236721,  0.06148892,  0.02948582,
        0.1175248 , -0.05198265,  0.04853149, -0.04528742, -0.01046432,
       -0.04674789,  0.10230402,  0.01338646,  0.07745194, -0.00323039,
       -0.03383607, -0.06252462, -0.06361134, -0.05356198, -0.00921301,
        0.08401891, -0.00691842,  0.03547987,  0.02727933,  0.09059671,
        0.02030631,  0.07606818,  0.0429274 , -0.02246193, -0.04

## Qdrant Client を初期化

In [19]:
from qdrant_client import QdrantClient

qdrant_url = "http://localhost:6333"
qdrant_collection_name = "livedoor_news"

# Qdrant Client を初期化
client = QdrantClient(url=qdrant_url)

In [20]:
from qdrant_client.models import Distance, VectorParams

# コレクション(テーブル)を初期化
# ベクトル次元 384次元、距離: cosine
# (既にコレクションが定義されている場合はエラーが出ます)
#  -> b'{"status":{"error":"Wrong input: Collection `livedoor_news` already exists!"},"time":0.0000995}'
client.create_collection(
    collection_name=qdrant_collection_name,
    vectors_config=VectorParams(size=embedding_dim, distance=Distance.COSINE),
)

True

In [21]:
# ペイロードのインデックス用にスキーマを定義
# None の場合はインデックスを作成しない
payload_index_field_types = {
    "id":"integer",  # URL から切り出した ID
    "url":None,  # 記事の URL
    "timestamp":"datetime",  # タイムスタンプ
    "label_name":"keyword",  # 記事区分名
    "label":"integer",  # 記事区分ID
    "text_length":"integer",  # テキストの文字列長
}

In [22]:
# ペイロード(メタデータ)の型を変換する: List[Dict[str, any]]
payloads = text_df[payload_index_field_types.keys()].to_dict('records')
payloads[0]

{'id': 5834709,
 'url': 'http://news.livedoor.com/article/detail/5834709/',
 'timestamp': '2011-09-04T10:00:00+0900',
 'label_name': 'dokujo-tsushin',
 'label': 0,
 'text_length': 1034}

In [23]:
from qdrant_client.models import PointStruct, Batch

# ポイント(アイテム)を追加
# ベクトルとペイロードを追加
operation_info = client.upsert(
    collection_name=qdrant_collection_name,
    wait=True,  # 検索可能な状態になるまで応答を待つオプション
    points=Batch(
        ids=text_df['id'].to_list(),
        payloads=payloads,
        vectors=vectors,
    )
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [24]:
# Qdrant の Payload にインデックスを設定
for field_name, field_type in payload_index_field_types.items():
    if field_type is None:
        continue
    print(f"field_name: {field_name}, field_type: {field_type}")
    client.create_payload_index(
        collection_name=qdrant_collection_name,
        field_name=field_name,
        field_schema=field_type,
    )

field_name: id, field_type: integer
field_name: timestamp, field_type: datetime
field_name: label_name, field_type: keyword
field_name: label, field_type: integer
field_name: text_length, field_type: integer


## 検索

In [25]:
# クエリ文字列からクエリベクトルを作成
query_text = 'インフラ関連の整備'
query_vector = model.encode(query_text)

In [26]:
# クエリベクトルの形
query_vector.shape

(384,)

In [27]:
# クエリベクトルに対するベクトル検索(Cosine類似度)
search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=query_vector,  # 検索キーベクトルを指定
    with_payload=True, # 応答にペイロードを含める
    limit=5
)

_ = [print(x) for x in search_result]

id=6611266 version=0 score=0.8272952 payload={'id': 6611266, 'label': 6, 'label_name': 'smax', 'text_length': 1445, 'timestamp': '2012-05-31T06:55:00+0900', 'url': 'http://news.livedoor.com/article/detail/6611266/'} vector=None shard_key=None
id=6066216 version=0 score=0.82396823 payload={'id': 6066216, 'label': 2, 'label_name': 'kaden-channel', 'text_length': 419, 'timestamp': '2011-11-27T16:30:00+0900', 'url': 'http://news.livedoor.com/article/detail/6066216/'} vector=None shard_key=None
id=6543512 version=0 score=0.82137966 payload={'id': 6543512, 'label': 6, 'label_name': 'smax', 'text_length': 2198, 'timestamp': '2012-05-09T22:38:00+0900', 'url': 'http://news.livedoor.com/article/detail/6543512/'} vector=None shard_key=None
id=6314562 version=0 score=0.81597924 payload={'id': 6314562, 'label': 2, 'label_name': 'kaden-channel', 'text_length': 496, 'timestamp': '2012-02-26T14:38:00+0900', 'url': 'http://news.livedoor.com/article/detail/6314562/'} vector=None shard_key=None
id=686597

In [28]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

# ベクトル検索 & フィルタリングの検索例
search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=query_vector,  # 検索キーベクトルを指定
    query_filter=Filter(
        must=[FieldCondition(key="label_name", match=MatchValue(value="it-life-hack"))]  # ペイロードの文字列一致でフィルタリング
    ),
    with_payload=True, # 応答にペイロードを含める
    limit=5,
)

_ = [print(x) for x in search_result]

id=6865979 version=0 score=0.8152049 payload={'id': 6865979, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 2413, 'timestamp': '2012-08-18T11:55:00+0900', 'url': 'http://news.livedoor.com/article/detail/6865979/'} vector=None shard_key=None
id=6409860 version=0 score=0.81104857 payload={'id': 6409860, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1098, 'timestamp': '2012-03-28T09:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6409860/'} vector=None shard_key=None
id=6509420 version=0 score=0.8105725 payload={'id': 6509420, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 868, 'timestamp': '2012-04-27T09:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6509420/'} vector=None shard_key=None
id=6574653 version=0 score=0.80989575 payload={'id': 6574653, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1171, 'timestamp': '2012-05-19T10:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6574653/'} vector=None shard_key=

In [29]:
from qdrant_client.models import Filter, FieldCondition, Range

# ベクトル検索 & フィルタリングの検索例
search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=query_vector,  # 検索キーベクトルを指定
    query_filter=Filter(
        must=[
            FieldCondition(
                key="text_length",
                range=Range(  # ペイロードの値の範囲でフィルタリング
                    gte=600,
                    lte=1000,
                ),
            ),
        ]
    ),
    with_payload=True, # 応答にペイロードを含める
    limit=10,
)

_ = [print(x) for x in search_result]

id=6509420 version=0 score=0.8105725 payload={'id': 6509420, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 868, 'timestamp': '2012-04-27T09:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6509420/'} vector=None shard_key=None
id=6737569 version=0 score=0.8075932 payload={'id': 6737569, 'label': 6, 'label_name': 'smax', 'text_length': 928, 'timestamp': '2012-07-09T07:55:00+0900', 'url': 'http://news.livedoor.com/article/detail/6737569/'} vector=None shard_key=None
id=6666132 version=0 score=0.80412245 payload={'id': 6666132, 'label': 2, 'label_name': 'kaden-channel', 'text_length': 880, 'timestamp': '2012-06-17T13:30:00+0900', 'url': 'http://news.livedoor.com/article/detail/6666132/'} vector=None shard_key=None
id=6711214 version=0 score=0.80290115 payload={'id': 6711214, 'label': 6, 'label_name': 'smax', 'text_length': 930, 'timestamp': '2012-07-01T09:55:00+0900', 'url': 'http://news.livedoor.com/article/detail/6711214/'} vector=None shard_key=None
id=5995609 ve

In [30]:

from qdrant_client.models import Filter, FieldCondition, MatchValue

# ベクトル検索 & フィルタリングの検索例
# 2つのフィルタリング条件を And で結合
search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=query_vector,  # 検索キーベクトルを指定
    query_filter=Filter(
        must=[
            FieldCondition(  # ペイロードの文字列一致でフィルタリング
                key="label_name",
                match=MatchValue(value="it-life-hack")
            ),
            FieldCondition(  # ペイロードの値の範囲でフィルタリング
                key="text_length",
                range=Range( gte=600, lte=1700,
                ),
            ),
        ]
    ),
    with_payload=True,  # ペイロードを取得
    limit=5,
)

_ = [print(x) for x in search_result]

id=6409860 version=0 score=0.81104857 payload={'id': 6409860, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1098, 'timestamp': '2012-03-28T09:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6409860/'} vector=None shard_key=None
id=6509420 version=0 score=0.8105725 payload={'id': 6509420, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 868, 'timestamp': '2012-04-27T09:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6509420/'} vector=None shard_key=None
id=6574653 version=0 score=0.80989575 payload={'id': 6574653, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1171, 'timestamp': '2012-05-19T10:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6574653/'} vector=None shard_key=None
id=6495338 version=0 score=0.80458814 payload={'id': 6495338, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1552, 'timestamp': '2012-04-23T15:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6495338/'} vector=None shard_key

In [31]:
# コレクションの情報を取得
client.get_collection( collection_name=qdrant_collection_name )

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=100, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), p

In [32]:
# コレクションを削除
# client.delete_collection( collection_name=qdrant_collection_name )

## まとめ

このセッションでは以下を確認しました。

- ベクトルとペイロード(メタデータ)の登録
  - Livedoor News コーパスの取得
  - SentenceTransfomers を利用した文章をベクトル化
  - ポイント(アイテム)の登録
  - ペイロードに対してインデックス設定
- 検索処理
  - ベクトル検索
  - フィルタリングとベクトル検索の併用